<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9710 sha256=16252b01c8aa3ba431ac35c4e66e1bada272fe43ef2b2ac51232dc0f72928118
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [10]:
import folium
import wget
import pandas as pd

In [11]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [12]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [13]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [16]:
import pandas as pd
import folium
from folium.features import DivIcon
import requests
from io import BytesIO

# Paso 1: Descargar y cargar el dataset con coordenadas
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_df = pd.read_csv(BytesIO(response.content))

# Paso 2: Filtrar columnas relevantes y obtener sitios únicos
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Paso 3: Centrar el mapa en el primer sitio o usar un promedio de coordenadas
center_lat = launch_sites_df['Lat'].mean()
center_long = launch_sites_df['Long'].mean()
site_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Paso 4: Añadir círculos y marcadores para cada sitio
for index, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    coordinate = [row['Lat'], row['Long']]
    
    # Círculo en el sitio
    circle = folium.Circle(
        location=coordinate,
        radius=100,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site_name))
    
    # Marcador con nombre
    marker = folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
    
    # Agregar al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostrar el mapa
site_map


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [17]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [18]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [19]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [22]:
# Task 2: Mark the success/failed launches for each site on the map
# Crear el mapa centrado como antes (usando el promedio de coordenadas)
center_lat = spacex_df['Lat'].mean()
center_long = spacex_df['Long'].mean()
site_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Agrupar por 'Launch Site', 'Lat', 'Long' y 'class' (éxito/fracaso)
launch_summary = spacex_df.groupby(['Launch Site', 'Lat', 'Long', 'class']).size().reset_index(name='Count')

# Pivotear para tener las columnas de éxitos y fracasos
launch_summary_pivot = launch_summary.pivot_table(index=['Launch Site', 'Lat', 'Long'],
                                                  columns='class',
                                                  values='Count',
                                                  fill_value=0).reset_index()

# Renombrar las columnas para mayor claridad
launch_summary_pivot.columns.name = None
launch_summary_pivot = launch_summary_pivot.rename(columns={0: 'Failed Launches', 1: 'Successful Launches'})

# Iterar sobre cada fila del dataframe pivotado
for index, row in launch_summary_pivot.iterrows():
    site_name = row['Launch Site']
    coordinate = [row['Lat'], row['Long']]
    successful_launches = row['Successful Launches']
    failed_launches = row['Failed Launches']

    # Determinar color basado en el número de lanzamientos exitosos y fallidos
    color = 'green' if successful_launches > 0 else 'red'

    # Crear el popup con la información total de lanzamientos
    popup_text = f"Site: {site_name} | Successful: {successful_launches} | Failed: {failed_launches}"

    # Crear marcador en el mapa
    folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=color),
        popup=popup_text
    ).add_to(site_map)

# Mostrar mapa con los marcadores
site_map

In [23]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [24]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [25]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Crear la columna marker_color en spacex_df basada en los valores de la columna 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verificar los primeros registros para asegurarnos de que la columna se creó correctamente
spacex_df.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [26]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [27]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
# Create a Marker object with its coordinate
# And customize the Marker's icon property to indicate if this launch was successful or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color'])

for index, record in spacex_df.iterrows():
    # Create a Marker object using the coordinates
    coordinate = [record['Lat'], record['Long']]
    
    # Customize the icon based on success or failure (use 'marker_color' for the color)
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Use marker_color to indicate success/failure
    )
    
    # Add the marker to the marker_cluster
    marker_cluster.add_child(marker)

# Display the map with the markers and clusters
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [28]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [29]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [34]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Función para calcular distancia entre dos puntos (Haversine)
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la Tierra en km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordenadas del sitio de lanzamiento (ejemplo: CCAFS SLC-40)
launch_site_lat = 28.56342
launch_site_lon = -80.56728

# Coordenadas del punto costero más cercano (ajusta según lo que encuentres en el mapa)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calcular la distancia
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distancia al punto costero más cercano: {distance_coastline:.2f} km")


Distancia al punto costero más cercano: 0.43 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [35]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from math import sin, cos, sqrt, atan2, radians
from folium.features import DivIcon

# Coordenadas del sitio de lanzamiento (ejemplo: CCAFS SLC-40)
launch_site_lat = 28.56342
launch_site_lon = -80.56728

# Coordenadas del punto costero más cercano (ajusta según lo que encuentres en el mapa)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Función para calcular distancia entre dos puntos (Haversine)
def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la Tierra en km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Calcular la distancia
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Crear el mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Añadir marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='CCAFS SLC-40 (Launch Site)',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Añadir marcador para el punto costero más cercano
folium.Marker(
    [coastline_lat, coastline_lon],
    popup='Punto Costero Más Cercano',
    icon=folium.Icon(color='green', icon='cloud')
).add_to(site_map)

# Añadir un marcador de distancia usando DivIcon
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
).add_to(site_map)

# Añadir una línea entre el sitio de lanzamiento y el punto costero
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='red',
    weight=2,
    opacity=0.7,
).add_to(site_map)

# Mostrar el mapa
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [37]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# site_map.add_child(lines)

import folium
from math import sin, cos, sqrt, atan2, radians
from folium.features import DivIcon

# Coordenadas del sitio de lanzamiento (ejemplo: CCAFS SLC-40)
launch_site_lat = 28.56342
launch_site_lon = -80.56728

# Coordenadas del punto costero más cercano (ajusta según lo que encuentres en el mapa)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Función para calcular distancia entre dos puntos (Haversine)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Radio aproximado de la Tierra en km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Calcular la distancia
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Crear el mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Añadir marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='CCAFS SLC-40 (Launch Site)',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Añadir marcador para el punto costero más cercano
folium.Marker(
    [coastline_lat, coastline_lon],
    popup='Punto Costero Más Cercano',
    icon=folium.Icon(color='green', icon='cloud')
).add_to(site_map)

# Añadir marcador de distancia usando DivIcon
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
).add_to(site_map)

# Crear la línea entre el sitio de lanzamiento y el punto costero
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2,
    color='red',
    opacity=0.8
)

# Añadir la línea al mapa
site_map.add_child(lines)

# Mostrar el mapa
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [38]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium
from math import sin, cos, sqrt, atan2, radians
from folium.features import DivIcon

# Coordenadas del sitio de lanzamiento (CCAFS SLC-40)
launch_site_lat = 28.56342
launch_site_lon = -80.56728

# Coordenadas del punto costero más cercano
coastline_lat = 28.56367
coastline_lon = -80.57163

# Coordenadas adicionales (ajusta según tus observaciones)
city_lat = 28.3922
city_lon = -80.6077

railway_lat = 28.5721
railway_lon = -80.5853

highway_lat = 28.5641
highway_lon = -80.5708

# Función para calcular distancia entre dos puntos (Haversine)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Radio aproximado de la Tierra en km
    lat1, lon1 = radians(lat1), radians(lon1)
    lat2, lon2 = radians(lat2), radians(lon2)
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Crear el mapa centrado en el sitio de lanzamiento
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=13)

# Añadir marcador para el sitio de lanzamiento
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='CCAFS SLC-40 (Launch Site)',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Lista de puntos de interés
points_of_interest = [
    {"name": "Coastline", "lat": coastline_lat, "lon": coastline_lon, "color": "green"},
    {"name": "City", "lat": city_lat, "lon": city_lon, "color": "orange"},
    {"name": "Railway", "lat": railway_lat, "lon": railway_lon, "color": "purple"},
    {"name": "Highway", "lat": highway_lat, "lon": highway_lon, "color": "gray"},
]

# Añadir marcadores y líneas para cada punto de interés
for poi in points_of_interest:
    # Calcular distancia
    distance = calculate_distance(launch_site_lat, launch_site_lon, poi["lat"], poi["lon"])

    # Añadir marcador
    folium.Marker(
        [poi["lat"], poi["lon"]],
        popup=f'{poi["name"]} ({distance:.2f} KM)',
        icon=folium.Icon(color=poi["color"], icon='info-sign')
    ).add_to(site_map)

    # Añadir marcador de distancia usando DivIcon
    folium.Marker(
        [poi["lat"], poi["lon"]],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance:.2f} KM</b></div>',
        )
    ).add_to(site_map)

    # Crear la línea entre el sitio de lanzamiento y el punto de interés
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [poi["lat"], poi["lon"]]],
        weight=2,
        color=poi["color"],
        opacity=0.8
    ).add_to(site_map)

# Mostrar el mapa
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
